# 0. Overview

# 1. Import Libraries and Constants

In [1]:
import numpy as np
import optuna

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

c:\Users\supha\repos\cs5756\RL-Improve-Promotion-Section\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
RAND_STATE = 42

# 2. Data Cleaning

In [3]:
# Mock dataset for demonstration
class TextDataset(Dataset):
    def __init__(self, texts, labels, sequence_length):
        self.texts = texts
        self.labels = labels
        self.sequence_length = sequence_length
        self.vocab = self.build_vocab(texts)
        self.encoded_texts = [self.encode_text(text) for text in texts]

    def build_vocab(self, texts):
        unique_words = set(word for text in texts for word in text.lower().split())
        vocab = {word: i + 1 for i, word in enumerate(unique_words)}  # +1 for padding token at index 0
        return vocab
    
    def encode_text(self, text):
        return [self.vocab.get(word, 0) for word in text.lower().split()][:self.sequence_length] + [0] * (self.sequence_length - len(text.split()))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded_texts[idx]), torch.tensor(self.labels[idx])
    
class DQN(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes, pre_trained_embeddings=None):
        super(DQN, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        if pre_trained_embeddings is not None:
            self.embedding.weight = nn.Parameter(pre_trained_embeddings)
            self.embedding.weight.requires_grad = False  # Or True if you want to fine-tune

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2) # Stacked LSTMs
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)  # Adjust for bidirectional LSTM
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        out = self.dropout(lstm_out[:, -1])
        out = self.relu(out)
        q_values = self.fc(out)
        return q_values

In [4]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # Using CrossEntropyLoss which combines LogSoftmax and NLLLoss
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def validate(model, device, validation_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += nn.CrossEntropyLoss()(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    validation_acc = correct / len(validation_loader.dataset)
    print(f'\nValidation set: Average loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} ({100. * correct / len(validation_loader.dataset):.0f}%)\n')
    return validation_loss, validation_acc

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [5]:
# Example texts and labels
texts = [
    "Great movie loved it", "Did not like the movie", "Awesome plot and acting",
    "Boring and too long", "Wonderfully made film", "Not my cup of tea",
    "Thrilling and exciting from start to finish", "Could have been better", "A masterpiece", "Waste of time",
    "The director did a fantastic job", "Terrible performances", "The cinematography is gorgeous", 
    "I fell asleep halfway through", "An unforgettable journey", "I've seen better", 
    "A total letdown from a great director", "The soundtrack complemented the movie perfectly", 
    "Lacked depth in the storyline", "Characters were not developed well", "Laughed all the way through", 
    "Failed to deliver a compelling narrative", "Visual effects were stunning", "Plot was predictable", 
    "Highly recommend this movie", "Not worth your time", "An epic disappointment", 
    "Engaging from start to finish", "Script was beautifully written", "The movie felt rushed",
    "A visual masterpiece", "Forgettable experience", "Will watch it again", "Do not waste your money", 
    "The acting was subpar", "A story well told", "Lacks originality", "Captivating to the very end", 
    "Missed the mark on many levels", "One of the year's best films"
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]

# Splitting dataset into train+val and test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=RAND_STATE)

# Splitting train+val into train and val
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_val_texts, train_val_labels, test_size=0.25, random_state=RAND_STATE)  # 0.25 x 0.8 = 0.2

# Creating datasets
sequence_length = 10  # Max number of words in a text
train_dataset = TextDataset(train_texts, train_labels, sequence_length)
validation_dataset = TextDataset(validation_texts, validation_labels, sequence_length)
test_dataset = TextDataset(test_texts, test_labels, sequence_length)

# Creating DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for data, label in train_loader:
    print(f"Train Encoded text: {data}")
    print(f"Train Label: {label}")
    break  # Just show one batch for brevity

for data, label in validation_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in test_loader:
    print(f"Test Encoded text: {data}")
    print(f"Test Label: {label}")
    break  # Just show one batch for brevity

Train Encoded text: tensor([[19, 75,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 8, 11, 38,  0,  0,  0,  0,  0,  0,  0],
        [62, 45, 64, 15,  0,  0,  0,  0,  0,  0],
        [52, 72, 76, 54,  0,  0,  0,  0,  0,  0]])
Train Label: tensor([1, 0, 1, 0])
Validation Encoded text: tensor([[24, 20,  1,  0,  0,  0,  0,  0,  0,  0],
        [12,  4, 25,  2,  0,  0,  0,  0,  0,  0],
        [12,  8,  9, 30, 14, 23,  0,  0,  0,  0],
        [29, 11, 15,  5,  0,  0,  0,  0,  0,  0]])
Validation Label: tensor([1, 0, 1, 1])
Test Encoded text: tensor([[17, 28, 18,  5, 20,  0,  0,  0,  0,  0],
        [31,  3,  1, 27, 31, 10, 12,  0,  0,  0],
        [13, 24, 21,  0,  0,  0,  0,  0,  0,  0],
        [25, 29, 30,  0,  0,  0,  0,  0,  0,  0]])
Test Label: tensor([0, 0, 0, 0])


In [6]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 10
learning_rate = 0.001

# Model, optimizer, and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    validate(model, device, validation_loader)

# After training, evaluate on the test set
test(model, device, test_loader)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.698385

Validation set: Average loss: 0.1960, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.616421

Validation set: Average loss: 0.2183, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.536101

Validation set: Average loss: 0.2374, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.593039

Validation set: Average loss: 0.2552, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.329406

Validation set: Average loss: 0.2610, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.952750

Validation set: Average loss: 0.2527, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.568822

Validation set: Average loss: 0.2515, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.634747

Validation set: Average loss: 0.2506, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.712374

Validation set: Average loss: 0.2491, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.532307

Validation set: Average loss: 0.2435, 

In [15]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 30
learning_rate = 0.001

def objective(trial):
    # Define the search space
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    embedding_dim = trial.suggest_categorical('embedding_dim', [64, 128, 256])
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])
    
    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Model setup with trial suggestions
    model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    epochs = 10  # Reduced for faster optimization cycles
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        val_loss, val_accuracy = validate(model, device, validation_loader)
    
    # Objective: maximize validation accuracy
    # print(f"Returning from validate: val_loss={val_loss}, val_accuracy={val_accuracy}")
    return val_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # Number of trials can be adjusted

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print(f'Value: {trial.value}')
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2024-03-31 15:03:49,334] A new study created in memory with name: no-name-cfe4661f-e59b-4b58-b0c9-3f4f0d41e176


Train Epoch: 1 [0/24 (0%)]	Loss: 0.700174

Validation set: Average loss: 0.6648, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.717181

Validation set: Average loss: 0.1734, Accuracy: 3/8 (38%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.609620

Validation set: Average loss: 0.3038, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.796684

Validation set: Average loss: 0.2835, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.408298

Validation set: Average loss: 0.3483, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.355392

Validation set: Average loss: 0.2482, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 1.136385

Validation set: Average loss: 0.2881, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.746285


[I 2024-03-31 15:03:50,481] Trial 0 finished with value: 0.45478782057762146 and parameters: {'learning_rate': 0.036421656584933876, 'embedding_dim': 128, 'hidden_dim': 128}. Best is trial 0 with value: 0.45478782057762146.



Validation set: Average loss: 0.4175, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 1.224056

Validation set: Average loss: 0.2360, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 1.044456

Validation set: Average loss: 0.4548, Accuracy: 1/8 (12%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.691070

Validation set: Average loss: 0.1792, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.678231

Validation set: Average loss: 0.1811, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.637699

Validation set: Average loss: 0.1831, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.677399

Validation set: Average loss: 0.1846, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.672430

Validation set: Average loss: 0.1860, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.634054

Validation set: Average loss: 0.1878, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.652585

Validation set: Average loss: 0.1896, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 

[I 2024-03-31 15:03:51,219] Trial 1 finished with value: 0.19553837925195694 and parameters: {'learning_rate': 0.00012331769793248198, 'embedding_dim': 128, 'hidden_dim': 32}. Best is trial 1 with value: 0.19553837925195694.


...

Train Epoch: 1 [0/24 (0%)]	Loss: 0.652790

Validation set: Average loss: 0.1721, Accuracy: 7/8 (88%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.670909

Validation set: Average loss: 0.1796, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.686842

Validation set: Average loss: 0.1933, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.651932

Validation set: Average loss: 0.2011, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.707075

Validation set: Average loss: 0.2076, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.710619

Validation set: Average loss: 0.2168, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.666870

Validation set: Average loss: 0.2256, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.725441

Validation set: Average loss: 0.2368, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.420478

Validation set: Average loss: 0.2506, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.595712


[I 2024-03-31 15:04:07,898] Trial 18 finished with value: 0.24915152788162231 and parameters: {'learning_rate': 0.00055462369822671, 'embedding_dim': 256, 'hidden_dim': 64}. Best is trial 11 with value: 0.17264394462108612.



Validation set: Average loss: 0.2492, Accuracy: 1/8 (12%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.696828

Validation set: Average loss: 0.1881, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.640605

Validation set: Average loss: 0.1892, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.656391

Validation set: Average loss: 0.1903, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.686512

Validation set: Average loss: 0.1912, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.640440

Validation set: Average loss: 0.1921, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.671763

Validation set: Average loss: 0.1929, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.691822

Validation set: Average loss: 0.1938, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.745802

Validation set: Average loss: 0.1947, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.650897


[I 2024-03-31 15:04:08,631] Trial 19 finished with value: 0.19716835767030716 and parameters: {'learning_rate': 6.341871230039989e-05, 'embedding_dim': 128, 'hidden_dim': 32}. Best is trial 11 with value: 0.17264394462108612.



Validation set: Average loss: 0.1961, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.654765

Validation set: Average loss: 0.1972, Accuracy: 1/8 (12%)

Number of finished trials: 20
Best trial:
Value: 0.17264394462108612
Params: 
    learning_rate: 1.3052514171811334e-05
    embedding_dim: 128
    hidden_dim: 64


In [11]:
study.optimize()

TypeError: Study.optimize() missing 1 required positional argument: 'func'

In [16]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_embedding_dim,params_hidden_dim,params_learning_rate,state
0,0,0.454788,2024-03-31 15:03:49.335227,2024-03-31 15:03:50.481947,0 days 00:00:01.146720,128,128,0.036422,COMPLETE
1,1,0.195538,2024-03-31 15:03:50.483223,2024-03-31 15:03:51.219960,0 days 00:00:00.736737,128,32,0.000123,COMPLETE
2,2,0.899254,2024-03-31 15:03:51.221466,2024-03-31 15:03:52.450398,0 days 00:00:01.228932,256,32,0.006990,COMPLETE
3,3,0.342020,2024-03-31 15:03:52.452399,2024-03-31 15:03:53.375609,0 days 00:00:00.923210,64,128,0.028913,COMPLETE
4,4,0.216977,2024-03-31 15:03:53.376637,2024-03-31 15:03:54.579927,0 days 00:00:01.203290,256,32,0.000322,COMPLETE
5,5,0.313953,2024-03-31 15:03:54.580929,2024-03-31 15:03:55.439669,0 days 00:00:00.858740,128,64,0.003544,COMPLETE
6,6,0.233942,2024-03-31 15:03:55.440670,2024-03-31 15:03:56.634240,0 days 00:00:01.193570,128,128,0.000121,COMPLETE
7,7,0.240924,2024-03-31 15:03:56.635246,2024-03-31 15:03:57.314988,0 days 00:00:00.679742,64,64,0.000348,COMPLETE
8,8,0.243491,2024-03-31 15:03:57.315990,2024-03-31 15:03:58.822325,0 days 00:00:01.506335,256,128,0.000200,COMPLETE
9,9,0.182972,2024-03-31 15:03:58.823348,2024-03-31 15:03:59.710697,0 days 00:00:00.887349,128,64,0.000021,COMPLETE
